In [1]:
%load_ext autoreload
%autoreload 2
#%matplotlib notebook

In [2]:
import sys
sys.path.append("../../odeon/")


In [3]:
import odeon
from odeon.models.opencd import OpenCDPlug

/home/PVoitot/mambaforge/envs/odeon/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [13]:
ocd = OpenCDPlug(
    config_file="../../open-cd/configs/changer/changer_ex_s50_512x512_40k_levircd.py",
    loss="ce"
)

10/26 11:05:15 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:40:32) [GCC 12.3.0]
    CUDA available: False
    numpy_random_seed: 499455131
    GCC: gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.0.0
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.1-Product Build 20220311 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=11.7, CUDNN_VERSION=8.5.0, CXX_COMPILER=/opt/rh/devtoolset-9/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=0 -fabi-version=11 -Wno-deprec

10/26 11:05:15 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
10/26 11:05:15 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                

In [26]:
# print(ocd.runner.build_optimizer(ocd.runner.optim_wrapper.optimizer))
ocd.runner.optim_wrapper = ocd.runner.build_optim_wrapper(ocd.config.optim_wrapper)
ocd.optimizer = ocd.runner.optim_wrapper.optimizer
ocd.scheduler = ocd.runner.build_param_scheduler(ocd.config.param_scheduler)

Executor open_cd_plug already exists. Will replace it


FileNotFoundError: [Errno 2] No such file or directory: 'data/LEVIR-CD/train/A'

In [5]:
levir_root = "/mnt/store_dai/datasrc/dchan/levir-cd-repatched-512x512"

import glob
from pathlib import Path
import csv
from matplotlib import pyplot as plt
#plt.ioff()

import odeon
from odeon.data.data_module import Input

levir_path = Path(levir_root)

batch_size = 10
num_workers = 8

train_params = {
    'input_fields': {
        "T0": {"name": "A", "type": "raster", "dtype": "uint8", "band_indices": [1, 2, 3], "mean": [0.485, 0.456, 0.406], "std": [0.228, 0.224, 0.225]},
        "T1": {"name": "B", "type": "raster", "dtype": "uint8", "band_indices": [1, 2, 3], "mean": [0.485, 0.456, 0.406], "std": [0.228, 0.224, 0.225]},
        "mask": {"name": "label", "type": "mask", "post_process": "floor_div", "dtype": "uint8"},
    },
    'dataloader_options' : {"batch_size": batch_size, "num_workers": num_workers},
    'input_file': levir_path / "train.csv",
    'input_files_has_header': 'infer',
    'root_dir': levir_path,
    'nb_samples': 0,
    'sample_seed': 0
}

val_params = {
    'input_fields': {
        "T0": {"name": "A", "type": "raster", "dtype": "uint8", "band_indices": [1, 2, 3], "mean": [0.485, 0.456, 0.406], "std": [0.228, 0.224, 0.225]},
        "T1": {"name": "B", "type": "raster", "dtype": "uint8", "band_indices": [1, 2, 3], "mean": [0.485, 0.456, 0.406], "std": [0.228, 0.224, 0.225]},
        "mask": {"name": "label", "type": "mask", "post_process": "floor_div", "dtype": "uint8"},       
    },
    'dataloader_options' : {"batch_size": batch_size, "num_workers": num_workers},
    'input_file': levir_path / "val.csv",
    'input_files_has_header': 'infer',
    'root_dir': levir_path,
    'nb_samples': 0,
    'sample_seed': 0
}

test_params = {
    'input_fields': {
        "T0": {"name": "A", "type": "raster", "dtype": "uint8", "band_indices": [1, 2, 3], "mean": [0.485, 0.456, 0.406], "std": [0.228, 0.224, 0.225]},
        "T1": {"name": "B", "type": "raster", "dtype": "uint8", "band_indices": [1, 2, 3], "mean": [0.485, 0.456, 0.406], "std": [0.228, 0.224, 0.225]},
        "mask": {"name": "label", "type": "mask", "post_process": "floor_div", "dtype": "uint8"},
    },
    'dataloader_options' : {"batch_size": batch_size, "num_workers": num_workers},
    'input_file': levir_path / "test.csv",
    'root_dir': levir_path,
    'input_files_has_header': 'infer',
    'nb_samples': 0,
    'sample_seed': 0
}


input = Input(
    fit_params=train_params,
    validate_params=val_params,
    test_params=test_params
)


In [6]:
input.prepare_data()
input.setup(stage="fit")

In [7]:
v = input.fit.dataset[0]
v

/home/PVoitot/mambaforge/envs/odeon/lib/python3.10/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


{'T1': tensor([[[-0.7413, -0.6119, -0.4017,  ...,  0.6737,  0.4393, -0.0135],
          [-0.6604, -0.7493, -0.7332,  ...,  0.0026,  0.0511,  0.1077],
          [-0.9030, -0.8706, -0.7170,  ..., -0.1672, -0.1591, -0.1106],
          ...,
          [-0.6281, -0.6200, -0.6200,  ..., -0.1833,  0.3341,  0.3584],
          [-0.5553, -0.5715, -0.5795,  ..., -0.2238,  0.0592,  0.3907],
          [-0.5310, -0.5715, -0.6038,  ..., -0.1995, -0.2076,  0.4069]],
 
         [[-0.6818, -0.5356, -0.3206,  ...,  0.7974,  0.5480,  0.0664],
          [-0.5958, -0.6818, -0.6732,  ...,  0.0836,  0.1352,  0.1954],
          [-0.8538, -0.8108, -0.6560,  ..., -0.0970, -0.0884, -0.0368],
          ...,
          [-0.5442, -0.5356, -0.5356,  ..., -0.1486,  0.4018,  0.4276],
          [-0.4668, -0.4840, -0.4926,  ..., -0.1916,  0.1094,  0.4620],
          [-0.4410, -0.4840, -0.5184,  ..., -0.1658, -0.1744,  0.4792]],
 
         [[-0.6040, -0.4784, -0.3045,  ...,  0.8159,  0.5358, -0.0051],
          [-0.5074, -0

In [8]:
import torch
r = ocd.model(torch.cat(tensors=(v["T0"].unsqueeze(0), v["T1"].unsqueeze(0)), dim=1))
r.shape

torch.Size([1, 2, 128, 128])

In [9]:
for idx, b in enumerate(input.fit.dataloader):
    print(b["mask"].unique())
    o = ocd.training_step(b, idx)
    print(o)
    if idx > 1:
        break

tensor([0., 1.], dtype=torch.float64)


/home/PVoitot/mambaforge/envs/odeon/lib/python3.10/site-packages/pytorch_lightning/core/module.py:416: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  rank_zero_warn(


{'loss': tensor(0.7431, grad_fn=<MulBackward0>)}
tensor([0., 1.], dtype=torch.float64)
{'loss': tensor(0.7461, grad_fn=<MulBackward0>)}
tensor([0., 1.], dtype=torch.float64)
{'loss': tensor(0.7514, grad_fn=<MulBackward0>)}


In [30]:
from typing import Optional, Dict
from albumentations.core.transforms_interface import DualTransform
import os


root_dir = "/mnt/store_dai/datasrc/dchan/gers/change/patches"
fold_nb: int = 0
    
def build_params(stage: str, root_dir: str, fold_nb: int, batch_size: int, num_workers: int, transform: Optional[DualTransform] = None, nb_samples: int = 0, sample_seed = 0) -> Dict:
    fold_dir: str = f'split-{fold_nb}'
    root_fold_dir: str = os.path.join(root_dir, fold_dir)
    dataset: str = os.path.join(root_fold_dir, f'{stage}_split_{fold_nb}.geojson')

    res = {
        'input_fields': {
            "T0": {"name": "T0", "type": "raster", "dtype": "uint8", "band_indices": [1, 2, 3]},
            "T1": {"name": "T1", "type": "raster", "dtype": "uint8", "band_indices": [1, 2, 3]},
            "mask": {"name": "change", "type": "mask", "encoding": "integer"}
        },
        'dataloader_options' : {"batch_size": batch_size, "num_workers": num_workers},
        'input_file': dataset,
        'root_dir': root_dir,
        'nb_samples': nb_samples,
        'sample_seed': sample_seed
    }

    if transform:
        res["transform"] = transform
    return res

nb_samples = 10
batch_size = 10
num_workers = 8
transform = None
seed = 0

fit_params = build_params("train", root_dir, fold_nb, batch_size, num_workers, transform, nb_samples, seed)
val_params = build_params("val", root_dir, fold_nb, batch_size, num_workers, None, nb_samples, seed)
test_params = build_params("test", root_dir, fold_nb, batch_size, num_workers, None, nb_samples, seed)

input_gers = Input(
        fit_params=fit_params,
        validate_params=val_params,
        test_params=test_params
    )

In [32]:
input_gers.prepare_data()
input_gers.setup(stage="fit")

Sampled                                                                               id  \
1694                       473296.0-6289428.800000001-473398.4-6289531.200000001   
3287   453788.80000000005-6316052.800000001-453891.20000000007-6316155.200000001   
1867  449436.80000000005-6290145.6000000015-449539.20000000007-6290248.000000002   
2161             457014.4-6298388.800000001-457116.80000000005-6298491.200000001   
3706           467766.40000000014-6319534.4-467868.80000000016-6319636.800000001   
3050  444316.80000000005-6311393.6000000015-444419.20000000007-6311496.000000002   
1808   443804.80000000016-6290043.200000001-443907.2000000002-6290145.6000000015   
716              458601.6000000001-6279086.4-458704.0000000001-6279188.800000001   
88      499049.6000000001-6260705.6000000015-499152.0000000001-6260808.000000002   
2017            453532.80000000016-6292654.4-453635.2000000002-6292756.800000001   

        id_zone  \
1694   CU_S1_90   
3287  VU_S1_191   
1867   CN_

In [34]:
v = input_gers.fit.dataset[0]
v

{'T1': tensor([[[-0.3216, -0.3451, -0.3608,  ..., -0.6118, -0.6157, -0.5882],
          [-0.3216, -0.3294, -0.3569,  ..., -0.6431, -0.6275, -0.5882],
          [-0.3529, -0.3373, -0.3373,  ..., -0.6353, -0.6157, -0.5765],
          ...,
          [-0.5882, -0.5922, -0.6000,  ..., -0.5725, -0.5725, -0.5686],
          [-0.6039, -0.5922, -0.5843,  ..., -0.5725, -0.5843, -0.5725],
          [-0.6039, -0.5922, -0.5765,  ..., -0.5686, -0.5725, -0.5686]],
 
         [[-0.3490, -0.3765, -0.4118,  ..., -0.5529, -0.5569, -0.5294],
          [-0.3451, -0.3686, -0.3961,  ..., -0.5843, -0.5686, -0.5294],
          [-0.3961, -0.3882, -0.3804,  ..., -0.5804, -0.5647, -0.5176],
          ...,
          [-0.5451, -0.5569, -0.5569,  ..., -0.5412, -0.5412, -0.5373],
          [-0.5608, -0.5490, -0.5490,  ..., -0.5412, -0.5529, -0.5529],
          [-0.5490, -0.5490, -0.5373,  ..., -0.5412, -0.5412, -0.5412]],
 
         [[-0.4275, -0.4431, -0.4627,  ..., -0.6941, -0.6824, -0.6627],
          [-0.4078, -0

In [ ]:
for idx, b in enumerate(input_gers.fit.dataloader):
    print(b["mask"].unique())
    o = ocd.training_step(b, idx)
    print(o)
    if idx > 1:
        break

Executor open_cd_plug already exists. Will replace it


tensor([0, 1], dtype=torch.uint8)
TOTO torch.Size([10, 2, 512, 512]) torch.Size([10, 512, 512]) torch.Size([10, 512, 512])
{'loss': tensor(0.7129, grad_fn=<NllLoss2DBackward0>)}
tensor([0, 1], dtype=torch.uint8)
